# Unsloth SFT

*https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Qwen3_(14B)-Reasoning-Conversational.ipynb#scrollTo=QmUBVEnvCDJv*

In [ ]:
!pip install -U --quiet ipywidgets

In [2]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [3]:
from unsloth import FastLanguageModel
import torch
from transformers import DataCollatorForLanguageModeling, TrainerCallback, DataCollatorForLanguageModeling
import pandas as pd
from datasets import Dataset, load_dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-13 09:27:24 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-13 09:27:24 [__init__.py:239] Automatically detected platform cuda.


In [1]:
import multiprocessing

# Save the original function
_original_cpu_count = multiprocessing.cpu_count

# Override it to return 1
def _patched_cpu_count():
    return 1

# Apply the patch
multiprocessing.cpu_count = _patched_cpu_count

In [19]:
del model
del tokenizer
#del trainer

In [67]:
import torch
torch.cuda.empty_cache()

In [5]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen3-0.6B",
    #model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = 4096,   # Context length - can be longer, but uses more memory
    load_in_4bit = True,     # 4bit uses much less memory
    load_in_8bit = False,    # A bit more accurate, uses 2x memory
    full_finetuning = False, # We have full finetuning now!
    # token = "hf_...",      # use one if using gated models
)

==((====))==  Unsloth 2025.5.1: Fast Qwen3 patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    NVIDIA A2. Num GPUs = 1. Max memory: 14.529 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/576M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/10.3k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/4.67k [00:00<?, ?B/s]

In [6]:
# !Important
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token
model.eval()

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024, padding_idx=151654)
    (layers): ModuleList(
      (0): Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear4bit(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear4bit(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Q

In [7]:
user_metadata = """
Title: Mémo visuel d'agronomie
Subtitle: enjeux et défis de l'agriculture expliqués en 100 infographies
Author: Benjamin Nowak (1988-....)
Publisher: Dunod
Place of publication: Malakoff
Year: DL 2024 (likely 2024)
Edition: Not explicitly stated
Material description: 1 vol. (207 p.); couverture illustrée en couleurs, illustrations en couleurs; 17 × 24 cm
ISBN: 978-2-10-086178-1
Other identifier: (OCoLC)1423791488; 9782100861781; FRBNF474231740000008
Language: French (fre)
Country of publication: FR (France)
Collection/Series: Not specified
Bibliographic references: Bibliography p. [200-204]. Glossaire. Index
Abstract/Notes: Cet ouvrage présente, sous une forme visuelle et synthétique, les notions indispensables à tout étudiant découvrant l'agronomie. À travers 100 infographies en couleurs, il leur permettra de s'initier aux enjeux économiques, scientifiques et environnementaux qui fonderont l'ensemble de leur cursus. Les plus : - 100 data visualisations commentées, pour une initiation synthétique à une science complexe. - Un panorama allant des débuts de l'agriculture aux enjeux de l'agronomie de demain. - Un chapitre permettant de s'initier à l'analyse des données et à la conception des infographies scientifiques.
Source of abstract/notes: éditeur
Keywords/Subjects:
- Agronomie (RAMEAU)
- Agriculture — Aspect économique (RAMEAU)
- Tableaux, graphiques, etc. (RAMEAU)
URL: http://catalogue.bnf.fr/ark:/12148/cb474231743
Cataloguing agencies: FR-751131015, Abes, AFNOR
Additional notes: Data fields indicate content as textual (txt), media as nonprojected media (n), carrier as unmediated (nga) per RDA codes indicated.
"""

messages = [
    {"role": "system", "content": "You are an expert bibliographic cataloger assistant. Convert metadata to UNIMARC/XML format."},
    {"role": "user", "content": f"Please convert this metadata to UNIMARC/XML:\n{user_metadata}"}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = True
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 4096, # Increase for longer outputs!
    temperature = 0.6, top_p = 0.95, top_k = 20, # For thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

<think>
Okay, let me start by understanding what the user is asking. They want to convert the provided metadata into UNIMARC XML. First, I need to parse the information from the given text. The metadata includes the title, subtitle, author, publisher, place of publication, year, edition, material description, ISBN, other identifiers, language, country, collection/series, bibliography, abstract/notes, keywords, and URL.

Now, I remember that UNIMARC XML follows a specific structure. The first part is the root element, then there's a series of elements. The title should be a <title> element. The subtitle is in <subtitle> and needs to be enclosed in <title> tags. The author and publisher are in <author> and <publisher>, respectively.

The material description includes the volume information: 1 vol. (207 p.), colors, illustrations, size. The ISBN is in <isbn> and the other identifiers are also in <other_identifier>. The language is French, and the country is France. The collection/series i

In [65]:
response = tokenizer.decode(_[0], skip_special_tokens=False)
format_reward(extract_xml(response))

0.0

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,           # Choose any number > 0! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,  # Best to choose alpha = rank or rank*2
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,   # We support rank stabilized LoRA
    loftq_config = None,  # And LoftQ
)

Unsloth 2025.5.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


## Tokenizer

## Data prep

In [6]:
from datasets import load_dataset
reasoning_dataset = load_dataset("Geraldine/metadata-to-unimarc-traces", split="train")
xml_dataset = load_dataset("Geraldine/Unimarc-iln050-5k", split="train")
xml_dataset = xml_dataset.select(range(200))

In [8]:
reasoning_dataset

Dataset({
    features: ['ppn', 'metadata', 'unimarc_record', 'reasoning'],
    num_rows: 673
})

In [9]:
xml_dataset

Dataset({
    features: ['ppn', 'question', 'answer'],
    num_rows: 200
})

In [7]:
system_prompt = "You are an expert bibliographic cataloger specializing in UNIMARC format"

def chat_templating_func(dataset):
    return tokenizer.apply_chat_template(
        dataset["messages"],
        tokenize = False,
    )

def create_reasoning_conversation(row):
  return {
    "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Please convert this metadata to UNIMARC/XML:\n{row['metadata']}"},
            {"role": "assistant", "content": f"<think>\n{row['reasoning']}\n</think>\n\n{row['unimarc_record']}"}
        ]
  }
reasoning_dataset = reasoning_dataset.map(create_reasoning_conversation, remove_columns=reasoning_dataset.features, batched=False)
reasoning_conv = chat_templating_func(reasoning_dataset)

def create_xml_conversation(row):
  return {
    "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Please convert this metadata to UNIMARC/XML:\n{row['question']}"},
            {"role": "assistant", "content": f"<think>\n\n</think>\n\n{row['answer']}"}
        ]
  }
xml_dataset = xml_dataset.map(create_xml_conversation, remove_columns=xml_dataset.features, batched=False)
xml_conv = chat_templating_func(xml_dataset)

data = pd.concat([
    pd.Series(reasoning_conv),
    pd.Series(xml_conv)
])
data.name = "text"
combined_dataset = Dataset.from_pandas(pd.DataFrame(data).reset_index())
combined_dataset

Dataset({
    features: ['index', 'text'],
    num_rows: 873
})

In [11]:
combined_dataset[0]["text"]

'<|im_start|>system\nYou are an expert bibliographic cataloger specializing in UNIMARC format<|im_end|>\n<|im_start|>user\nPlease convert this metadata to UNIMARC/XML:\nTitle: Yoga senior  \nSubtitle: la méthode de Gasquet  \nAuthor: Dr Bernadette de Gasquet  \nParticipation: avec la participation du Dr Marie Thirion  \nAuthor details:  \n- Bernadette de Gasquet (born 1946)  \n- Marie Thirion (born 1944), pediatrician  \n\nPublisher: Robert Laffont  \nPlace of publication: Paris  \nYear: DL 2019 (Legal deposit 2019)  \n\nISBN: 978-2-221-24174-5  \nOther identifier: 9782221241745 (EAN)  \nOCLC number: 1107377985  \n\nLanguage: French (fre)  \n\nEdition: Not explicitly stated; appears first edition  \n\nMaterial Description:  \n- 1 volume (250 pages)  \n- Illustrations, color illustrated cover  \n- Dimensions: 22 cm  \n\nCollection/Series: Réponses  \n\nNotes/Bibliographic references: Notes bibliogr.  \n\nAbstract/Notes:  \nLa méthode de Gasquet enfin adaptée aux seniors ! Quelle activit

## Training

In [8]:
# Split dataset
train_size = int(len(combined_dataset) * 0.9)
train_dataset = combined_dataset.select(range(train_size))
eval_dataset = combined_dataset.select(range(train_size, len(combined_dataset)))

In [ ]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=104): 100%|██████████| 785/785 [00:40<00:00, 19.31 examples/s]
num_proc must be <= 88. Reducing num_proc to 88 for dataset of size 88.
Unsloth: Tokenizing ["text"] (num_proc=88):  27%|██▋       | 24/88 [00:11<00:27,  2.30 examples/s]

In [13]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 873 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 20,185,088/6,000,000,000 (0.34% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.585700
2,1.585300
3,1.492200
4,1.496300
5,1.456100
6,1.371700
7,1.500300
8,1.404000
9,1.344200
10,1.288800


In [ ]:
user_metadata = """
Title: La dynastie des Ming
Author: Jean-Pierre Duteil
Editor/Director: sous la direction de Jean Heuclin
Author details:
- Duteil, Jean-Pierre (1952-...)
- Heuclin, Jean (1949-...), historien
Edition: Nouvelle édition
Publisher: Ellipses
Place of publication: Paris
Year: 2024
ISBN: 978-2-340-08969-3 (note: also 9782340089693)
Language: French
Country of publication: France
Physical description: 1 volume (287 pages), includes cartes, plans, cover illustrated in color, 18 cm
Bibliography: Bibliogr. p. 279-286
Abstract/Notes:
Les Ming, une dynastie brillante dominée par les Han installés à Pékin à l’époque du Quattrocento et de la Renaissance en Europe ; les deux civilisations devaient se rencontrer par une expansion maritime similaire, une découverte inquiète et remplie des mystères orientaux. Le fondateur Hongwu (1368-1398) à l’instar de notre Henri IV, tournant le dos à l’influence nomade mongole, établit une société rurale autosuffisante dont l’essor des productions céréalières alimenta toute une économie marchande et urbaine. Des projets grandioses virent le jour comme le Grand Canal, la Grande Muraille ou la Cité interdite avec une puissante marine et une armée de métier. Tout un système administratif de fonctionnaires recrutés par concours sur leurs mérites accompagne une remarquable création littéraire, calligraphique et artistique facilitée par l’impression de livres. Cette magnifique construction tomba progressivement sous l’emprise sclérosante de ces mêmes « lettrés » trop empreints d’un confucianisme qui résistait mal à la critique de la politique gouvernementale et au comportement extravagant des empereurs. Des révoltes se firent jour durant le dernier siècle de la dynastie marquée par une succession de catastrophes naturelles et d’épidémies qui ruinèrent l’une des périodes les plus riches de l’histoire de l’Empire du Milieu.
Source of the abstract/notes: 4e de couverture (back cover)
Subject/Keywords:
- Chine
- Dynastie des Ming (1368-1644)
Classification: 951.02 (subject classification)
Publication history note: Paru précédemment en 2016
Additional edition note: La 1ère édition de ce livre a été publiée en grand format en 2016 dans la collection "Biographies et mythes historiques" dirigée par Stéphane Curveiller
Collection/Series: Collection "Biographies et mythes historiques" (inferred from note about first edition)
"""

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": f"Please convert this metadata to UNIMARC/XML:\n{user_metadata}"},
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = True
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 4096, # Increase for longer outputs!
    temperature = 0.6, top_p = 0.95, top_k = 20, # For thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

# Unsloth GRPO

*https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Phi_4_(14B)-GRPO.ipynb#scrollTo=DkIvEkIIkEyB*

In [2]:
!pip install -U --quiet fuzzywuzzy unsloth vllm

In [21]:
del trainer

In [23]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
max_seq_length = 4096 # Can increase for longer reasoning traces
lora_rank = 16 # Larger rank = smarter, but slower


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen3-0.6B",
    #model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = max_seq_length,   # Context length - can be longer, but uses more memory
    load_in_4bit = True,     # 4bit uses much less memory
    load_in_8bit = False,    # A bit more accurate, uses 2x memory
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.7,      # use one if using gated models
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

# !Important
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token

Unsloth: Switching from Unsloth dynamic quant to normal quant since
we do not yet support fast inference for unsloth/qwen3-0.6b-unsloth-bnb-4bit
==((====))==  Unsloth 2025.5.1: Fast Qwen3 patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    NVIDIA A2. Num GPUs = 1. Max memory: 14.529 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Your GPU cannot handle sequence lengths of 256 due to limited GPU memory.
Unsloth: Your GPU can only handle approximately the maximum sequence length of 256.
Unsloth: vLLM loading unsloth/qwen3-0.6b-bnb-4bit with actual GPU utilization = 3.0%
Unsloth: Your GPU has CUDA compute capability 8.6 with VRAM = 14.53 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill t

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]



Unsloth: Retrying vLLM to process 96 sequences and 256 tokens in tandem.
Error:
CUDA out of memory. Tried to allocate 304.00 MiB. GPU 0 has a total capacity of 14.53 GiB of which 174.00 MiB is free. Process 3968688 has 14.35 GiB memory in use. Of the allocated memory 13.11 GiB is allocated by PyTorch, with 37.88 MiB allocated in private pools (e.g., CUDA Graphs), and 64.60 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
INFO 05-13 13:18:33 [config.py:717] This model supports multiple tasks: {'embed', 'generate', 'classify', 'score', 'reward'}. Defaulting to 'generate'.
INFO 05-13 13:18:33 [config.py:2003] Chunked prefill is enabled with max_num_batched_tokens=256.
Unsloth: vLLM Bitsandbytes config using kwargs = {'load_in_8bit': False, 'load_in_4bit':

RuntimeError: CUDA out of memory. Tried to allocate 304.00 MiB. GPU 0 has a total capacity of 14.53 GiB of which 174.00 MiB is free. Process 3968688 has 14.35 GiB memory in use. Of the allocated memory 13.11 GiB is allocated by PyTorch, with 37.88 MiB allocated in private pools (e.g., CUDA Graphs), and 64.60 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [12]:
import xml.etree.ElementTree as ET
import re
import difflib
from fuzzywuzzy import fuzz
from inflection import singularize

def extract_hash_answer(text: str) -> str | None:
    # Use model response structure to find the XML part
    if "</think>" not in text:
        return text
    return text.split("</think>")[1].strip()

def extract_xml(text: str) -> str | None:
    # Use regular expression to find the XML part enclosed in ```xml...```
    xml_match = re.search(r'```xml(.*?)```', text, re.DOTALL)
    if xml_match:
        return xml_match.group(1).strip()
    else:
        return None

def extract_field_values(xml_str):
    root = ET.fromstring(xml_str)
    fields = {}
    for df in root.findall(".//datafield"):
        tag = df.get("tag")
        subfields = [sf.text for sf in df.findall("subfield")]
        fields[tag] = " ".join(subfields)
    return fields

def parse_metadata(user_input: str) -> dict:
    """
    Parses textual bibliographic data into a dictionary.
    """
    metadata = {}
    lines = user_input.strip().split('\n')
    for line in lines:
        if ":" in line:
            key, value = line.split(":", 1)
            metadata[key.strip()] = value.strip()
    return metadata

def format_reward(xml_output: str) -> float:
    if xml_output is None:
        return 0.0
    try:
        root = ET.fromstring(xml_output)
    except ET.ParseError:
        return 0.0

    score = 0.2  # Starts above 0 for well-formed but noncompliant XML

    if root.find(".//controlfield") is not None:
        score += 0.1
    if root.find(".//leader") is not None:
        score += 0.1
    if root.find(".//datafield") is not None:
        score += 0.3
    if root.find(".//subfield") is not None:
        score += 0.3

    return min(score, 1.0)

def accuracy_reward(generated_xml: str, target_xml: str) -> float:
    if generated_xml is None or target_xml is None:
        return 0.0
    try:
        gen_fields = extract_field_values(generated_xml)
        tgt_fields = extract_field_values(target_xml)
    except ET.ParseError:
        return 0.0

    shared_keys = set(gen_fields) & set(tgt_fields)
    if not shared_keys:
        return 0.0

    total_sim = 0
    for key in shared_keys:
        sim = difflib.SequenceMatcher(None, gen_fields[key], tgt_fields[key]).ratio()
        total_sim += sim

    return total_sim / len(shared_keys)

def semantic_field_reward(user_prompt: str, generated_xml: str) -> float:
    if generated_xml is None or user_prompt is None:
        return 0.0
    FIELD_MAPPINGS = {
        "Title": ("200", "a"),              # Title → 200$a
        "Subtitle": ("200", "e"),           # Subtitle → 200$e
        "Year": ("214", "d"),               # Year → 214$d (Publication date)
        "Authors": ("700", "a"),            # Authors → 700$a (Main Author)
        "Publisher": ("214", "c"),          # Publisher → 214$c
        "ISBN": ("010", "a"),               # ISBN → 010$a
        "Language": ("101", "a"),           # Language → 101$a
        "Collection/Series": ("225", "a"),         # Collection/Series → 225$a
        "Material description": ("215", "a"), # Material description → 215$a
        "Price": ("010", "d"),              # Price → 010$d
        "Abstract/Notes Source": ("330", "a"),           # Abstract → 330$a
        "Country of publication": ("102", "a"),            # Country of publication → 102$a
        "Edition": ("205", "a"),            # Edition → 205$a
        "Notes": ("300", "a"),              # Notes → 300$a
        "Keywords / Subject Headings": ("600", "a")            # Keywords → 600$a (Subject Headings)
    }
    try:
        root = ET.fromstring(generated_xml)
    except ET.ParseError:
        return 0.0
        metadata = parse_metadata(user_prompt)
    total_possible_matches = len(FIELD_MAPPINGS)
    successful_matches = 0

    for input_field, input_value in metadata.items():
        best_match_score = 0
        best_matched_unimarc = None

        for unimarc_field_label, (tag, subfield) in FIELD_MAPPINGS.items():
            similarity_score = fuzz.partial_ratio(input_field.lower(), unimarc_field_label.lower())
            if similarity_score > best_match_score and similarity_score > 80: # Adjust threshold
                best_match_score = similarity_score
                best_matched_unimarc = (tag, subfield)

        if best_matched_unimarc:
            tag, subfield = best_matched_unimarc
            datafield = root.find(f".//datafield[@tag='{tag}']/subfield[@code='{subfield}']")
            if datafield is not None:
                generated_value = datafield.text.strip().lower()
                metadata_value = input_value.strip().lower()
                if metadata_value in generated_value or generated_value in metadata_value:
                    successful_matches += 1

    reward = successful_matches / total_possible_matches if total_possible_matches > 0 else 0.0
    return round(reward, 2)

def format_reward_func(completions, **kwargs) -> list[float]:
    rewards = []
    for completion in completions:
        response = completion[0]['content']
        extracted_xml = extract_xml(response)  # Now returns a single XML string or None
        rewards.append(format_reward(extracted_xml))
    return rewards

def accuracy_reward_func(completions, answer, **kwargs) -> list[float]:
    rewards = []
    for completion, target in zip(completions, answer):
        response = completion[0]['content']
        extracted_xml = extract_xml(response)  # Returns the first XML string or None
        rewards.append(accuracy_reward(extracted_xml, target))
    return rewards

def semantic_field_reward_func(prompts, completions, **kwargs) -> list[float]:
    rewards = []
    for prompt_group, completion in zip(prompts, completions):
        # Assuming your prompt is the last turn in the conversation history
        user_prompt = prompt_group[-1]['content']
        response = completion[0]['content']
        extracted_xml = extract_xml(response)

        if extracted_xml:
            reward = semantic_field_reward(user_prompt, extracted_xml)
            rewards.append(reward)
        else:
            rewards.append(0.0)
    return rewards

In [16]:
from datasets import load_dataset, Dataset

def get_dataset(split = "train") -> Dataset:
    data = load_dataset("Geraldine/metadata-to-unimarc-traces", split=split)
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': "You are an expert bibliographic cataloger assistant. Convert metadata to UNIMARC/XML format"},
            {'role': 'user', 'content': f"Please convert this metadata to UNIMARC/XML:\n{x['metadata']}"}
        ],
        'answer': x['unimarc_record']
    }) # type: ignore
    return data # type: ignore
dataset = get_dataset()
dataset = dataset.select(range(50))

In [22]:
#del trainer
import torch
torch.cuda.empty_cache() 

In [17]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 6, # Decrease if out of memory
    max_prompt_length = 2048,
    max_completion_length = max_seq_length,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 100,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
    do_sample=True,
    top_p=0.9,
    top_k=50,
    repetition_penalty=1.2,
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 6


In [18]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        format_reward_func,
        accuracy_reward_func,
        semantic_field_reward_func
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 50 | Num Epochs = 2 | Total steps = 100
O^O/ \_/ \    Batch size per device = 6 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (6 x 1 x 1) = 6
 "-____-"     Trainable parameters = 10,092,544/6,000,000,000 (0.17% trained)


Step,Training Loss,reward,reward_std,completion_length,kl,rewards / format_reward_func_2,rewards / accuracy_reward_func_2
1,0.000000,0.033333,0.081650,1135.166748,0.000000,0.033333,0.000000
2,0.000000,0.066667,0.103280,1047.500000,0.000000,0.066667,0.000000
3,0.000000,0.000000,0.000000,910.833374,0.000601,0.000000,0.000000
4,0.000000,0.066667,0.103280,1121.666748,0.000582,0.066667,0.000000
5,0.000000,0.066667,0.103280,826.666687,0.000587,0.066667,0.000000
6,0.000000,0.100000,0.109545,1367.666748,0.000631,0.100000,0.000000
7,0.000000,0.100000,0.109545,1199.500000,0.000710,0.100000,0.000000
8,0.000000,0.000000,0.000000,1319.333374,0.000568,0.000000,0.000000


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.12 GiB. GPU 0 has a total capacity of 14.53 GiB of which 638.00 MiB is free. Process 3968688 has 13.89 GiB memory in use. Of the allocated memory 12.60 GiB is allocated by PyTorch, with 37.88 MiB allocated in private pools (e.g., CUDA Graphs), and 115.35 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [60]:
extracted_responses = ['<!--\nUNIMARC XML format for "Au nom des requins" (French) - 2022 by Actes Sud\n--> \n\n<notes>\n  <dedication> Dédicace de l\'auteur </dedication>\n  <statement> Includes bibliography and bibliographical notes. </statement>\n  <collection> \n    <title>Mondes sauvages</title> \n    <subtitle>pour une nouvelle alliance</subtitle>\n  </collection>\n  <collection/series>\n  </collection>\n  <bibs>\n    <bibliographique>\n      <note> Buteur: 289 </note>\n      <note> 4e de couv. </note>\n    </bibs>\n  </bibs>\n  <identifier>\n    <identifierType>isbn</identifierType>\n    <identifierValue>978-2-330-16034-0</identifierValue>\n  </identifier>\n  <identifier>\n    <identifierType>oclc</identifierType>\n    <identifierValue>1304465969</identifierValue>\n  </identifier>\n  <identifier>\n    <identifierType>issn</identifierType>\n    <identifierValue>2558-2682</identifierValue>\n  </identifier>\n  <bibsNote>\n    <note> Notes bibliographiques </note>\n  </bibsNote>\n  <keywords>\n    <keyword>Requins </keyword>\n    <keyword>Requins / Comportement animal </keyword>\n    <keyword>Éthologie </keyword>\n    <keyword>Relations homme-animal </keyword>\n  </keywords>\n</notes>\n\n<identifier>\n  <identifierType>isbn</identifierType>\n  <identifierValue>978-2330160340</identifierValue>\n</identifier>\n\n<material>\n  <materialCategory>text </materialCategory>\n  <materialLevel>txt </materialLevel>\n  <materialNote> media: non-projectable </materialNote>\n</material>', '<unimarc>\n  <title>Nom des requins</title>\n  <author>\n    <author>\n      <name>François Sarano</name>\n    </author>\n  </title>\n  <publisher>\n    <publisher>Actes Sud</publisher>\n  </publisher>\n  <preface>\n    <preface>\n      <name>Sandra Bessudo</name>\n    </preface>\n  </preface>\n  <illustrations>\n    <illustration>\n      <image>\n        <title>Marion Sarano</title>\n        <author>\n          <name>Marion Sarano</name>\n        </author>\n      </image>\n    </illustration>\n  </illustrations>\n  <illustrations>\n    <illustration>\n      <title>Map</title>\n      <author>\n        <name>1</name>\n      </author>\n    </illustration>\n  </illustrations>\n  <publisher>\n    <publisher>Arles</publisher>\n  </publisher>\n  <edition>\n    <title>DL 2022</title>\n  </edition>\n  <price>\n    <price>21 EUR</price>\n  </price>\n  <language>\n    <language>French</language>\n  </language>\n  <country_ofPublication>\n    <country>France</country>\n  </country_ofPublication>\n  <collection>\n    <digitalCollection>\n      <title>Mondes sauvages</title>\n      <description>pour une nouvelle alliance</description>\n    </digitalCollection>\n  </collection>\n  <physicalDescription>\n    <physicalDescription>\n      <title>1 volume</title>\n      <pageCount>291</pageCount>\n      <height>22 cm</height>\n      <illustrations>\n        <illustration>\n          <title>Illustrations</title>\n          <author>\n            <name>Marion Sarano</name>\n          </author>\n        </illustration>\n        <title>Map</title>\n        <author>\n          <title>1</title>\n        </author>\n      </illustrations>\n    </physicalDescription>\n  </physicalDescription>\n  <bibliographicNotes>\n    <bibliographicNotes>\n      <bibliographicNote>[[289]]</bibliographicNote>\n      <note>Bibliography</note>\n      <note>Notes bibliographiques</note>\n    </bibliographicNotes>\n  </bibliographicNotes>\n  <dedication>\n    <dedication>De l\'auteur</dedication>\n  </dedication>\n  <subject>\n    <subject>\n      <subject>\n        <title>Requins</title>\n      </subject>\n    </subject>\n  </subject>\n  <contributors>\n    <contributor>\n      <contributor>\n        <name>François Sarano</name>\n      </contributor>\n    </contributor>\n    <contributor>\n      <contributor>\n        <name>Marion Sarano</name>\n      </contributor>\n    </contributor>\n    <contributor>\n      <contributor>\n        <name>Sandra Bessudo</name>\n      </contributor>\n    </contributor>\n  </contributors>\n  <additionalIdentifiers>\n    <additionalIdentifier>\n      <identifier>OCLC number:1304465969</identifier>\n    </additionalIdentifier>\n    <additionalIdentifier>\n      <identifier> ISSN related to series "Mondes sauvages":2558-2682</identifier>\n    </additionalIdentifier>\n    <additionalIdentifier>\n      <identifier>Other identifier: 9782330160340</identifier>\n    </additionalIdentifier>\n  </additionalIdentifiers>\n  <materialType>\n    <materialType>\n      <title>text</title>\n      <medium>\n        <title>media</title>\n        <description>non-projectable</description>\n      </medium>\n    </materialType>\n  </materialType>\n  <sourceDatabase>\n    <sourceDatabase>\n      <title>Sudoc</title>\n    </sourceDatabase>\n  </sourceDatabase>\n  <catalogingAgency>\n    <catalogingAgency>\n      <title>AFNOR</title>\n    </catalogingAgency>\n  </catalogingAgency>\n  <recordCreationDate>\n    <recordCreationDate>2023-04-12</recordCreationDate>\n  </recordCreationDate>\n  <note>\n    <note>\n      <note>Bibliographic</note>\n      <note>Part of the collection "Mondes sauvages"</note>\n      <note>Includes bibliography...</note>\n    </note>\n    <note>Bibliographique</note>\n    <note>Dedication</note>\n    <note>Abstract/Description</note>\n    <note>4e de couv.</note>\n  </note>\n</unimarc>', "<unimarc:bookInfo>\n  <title>Avant-Garde des Requins</title>\n  <author>François Sarano</author>\n  <author>Sandra Bessudo</author>\n  <collectionSeries>Mondes sauvages;</collectionSeries>\n  <publisher>Actes Sud</publisher>\n  <place>Arles</place>\n  <date>2022</date>\n  <edition>DL 2022</edition>\n  <isbnNumber>978-2330160340</isbnNumber>\n  <price>21 EUR</price>\n  <language>French</language>\n  <country>France</country>\n  <collection>Sudoc</collection>\n</unimarc:bookInfo>\n\n<unimarc:abstract>\n  <text>\n    <description>20 mètres de profondeur. L'eau bleue sombre est peuplée de plancton. Face à moi, Lady Mystery, une énorme femelle requin blanc, soeur des « Dents de la mer » : 5,5 mètres, une tonne et demie. Puissance extrême que rien ne peut arrêter. Scientifique, je ne me laisse pas distraire : je consigne profondeur, heure, sexe et taille. Et soudain, à quelques mètres de l'œil qui me fixe, je réalise le dérisoire de ces informations, si réductrices qu'elles trahissent la créature indomptée que je cherche à connaître. Comment raconter cette élégance sauvage ? Comment traduire ce que ses sens, profondément différents des nôtres, lui disent de cette rencontre et de l'océan qui nous entoure ? Je me coule contre son flanc. Nous nageons épaule contre nageoire. La distance qui nous sépare ne se mesure pas en centimètre, elle se mesure en confiance réciproque. Minute d'éternité. Nous ne faisons qu'un corps. Je suis en paix. Rencontre authentique, sans calcul, qui procure la joie profonde de communier avec la vie.</description>\n  </text>\n</unimarc:abstract>\n\n<unimarc:note>\n  <bibliography><p><strong>Bibliographique</strong>:</p><ul><li><strong>289</strong> pages</li></ul><ul><li><strong>Notes bibliographiques</strong>:</li><ul><li><strong>[[4e de couv.]]</strong></li></ul></p>\n</unimarc:note>\n\n<unimarc:author>\n  <authorItem>\n    <author>François Sarano</authorItem>\n    <attribut> born: 1954 </attribut>\n  </authorItem>\n  <authorItem>\n    <author>Marion Sarano</authorItem>\n    <attribut> born: 1988 </attribut>\n  </authorItem>\n  <authorItem>\n    <author>Sandra Bessudo</authorItem>\n    <attribut> born: 1969 </attribut>\n  </authorItem>\n</unimarc:author>\n</unimarc:bookInfo>", "<unimarc:metadata>\n  <title>Amour des requins</title>\n  <author>François Sarano</author>\n  <preface>Sandra Bessudo</preface>\n  <illustrations>Marion Sarano (original illustrations)</illustrations>\n  <publisher>Actes Sud</publisher>\n  <place>Arles</place>\n  <year>2022</year>\n  <edition>DL 2022</edition>\n  <isbn>978-2330160340</isbn>\n  <price>21 EUR</price>\n  <language>French</language>\n  <country>France</country>\n  <collection/series>Mondes sauvages; subtitle: pour une nouvelle alliance</collection/series>\n  <physical description>1 volume (291 pages); illustrations, 1 map, color illustrated cover; 22 cm</physical description>\n  <bibliography>Includes bibliography and bibliographical notes</bibliography>\n  <dedication>Dédicace de l'auteur</dedication>\n  <keywords>Requins (Sharks)</keywords>\n  <contributrices>\n    <contributeur> François Sarano</contributeur>\n    <contributeur> Marion Sarano</contributeur>\n    <contributeur> Sandra Bessudo</contributeur>\n  </contributrices>\n  <additional identifiers>\n    <oclc>1304465969</oclc>\n    <issn>2558-2682</issn>\n    <other identifier>9782330160340</other identifier>\n  </additional identifiers>\n  <material type>text (rdacontent: txt), media: non-projectable (n)</material type>\n  <source database>Sudoc (French union catalogue)</source database>\n  <cataloging agency>AFNOR (French Association for Standardization)</cataloging agency>\n  <record creation date>2023-04-12</record creation date>\n</unimarc:metadata>", None, '<unimarc xmlns="http://unims.marca-iso.org/v3.24.0">\n  <series>\n    <collection>\n      <title>Marine Fishes</title>\n      <description>\n        Louis de la Courrière, cible ; collection « Mondes sauvages », sub-titres : « Pour une nouvelle alliance ».\n      </description>\n      <series-number>2</series-number>\n      <edition>2022</edition>\n      <publication>\n        <publisher>Actes Sud</publisher>\n        <publication>Arles</publication>\n        <year>2022</year>\n        <edition>DL 2022</edition>\n        <isbn>978-2-330-16034-0</isbn>\n        <price>21 EUR</price>\n        <language>French</language>\n        <country>France</country>\n      </publication>\n    </collection>\n  </series>\n  <contributors>\n    <contributor>\n      <author> François Sarano </author>\n      <birth>1954</birth>\n      <language>French</language>\n    </contributor>\n    <contributor>\n      <author> Sandra Bessudo </author>\n      <birth>1969</birth>\n    </contributor>\n    <contributor>\n      <illustrator> Marion Sarano </illustrator>\n      <birth>1988</birth>\n    </contributor>\n  </contributors>\n  <abstract>\n    <note> 4e de couv. </note>\n    <title>\n      "Vingt mètres de profondeur. L\'eau bleu sombre est peuplée de plancton. Face à moi, Lady Mystery, une énorme femelle requin blanc, soeur des « Dents de la mer » : 5,5 mètres, une tonne et demie. Puissance extrême que rien ne peut arrêter. Scientifique, je ne me laisse pas distraire : je consigne profondeur, heure, sexe et taille. Et soudain, à quelques mètres de l’œil qui me fixe, je réalise le dérisoire de ces informations, si réductrices qu’elles trahissent la créature indomptée que je cherche à connaître. Comment raconter cette élégance sauvage ? Comment traduire ce que ses sens, profondément différents des nôtres, lui disent de cette rencontre et de l’océan qui nous entoure ? Je me coule contre son flanc. Nous nageons épaule contre nageoire. La distance qui nous sépare ne se mesure pas en centimètre, elle se mesure en confiance réciproque. Minute d’éternité. Nous ne faisons qu’un corps. Je suis en paix. Rencontre authentique, sans calcul, qui procure la joie profonde de communier avec la vie." </title>\n  </abstract>\n  <keywords>\n    <keyword>Requins</keyword>\n    <keyword>Requins / Comportement animal (animal behavior)</keyword>\n    <keyword>Éthologie</keyword>\n    <keyword>Relations homme-animal</keyword>\n  </keywords>\n</unimarc>'][5]
format_reward_func_2(extracted_responses)

str

## Archives

In [8]:
def check_tokenization(dataset, tokenizer, num_samples=3):
    for i in range(min(num_samples, len(dataset))):
        # Get example
        example = dataset[i]
        
        # Extract assistant message
        assistant_msg = [msg["content"] for msg in example["messages"] if msg["role"] == "assistant"][0]
        
        # Check if reasoning markers appear
        reasoning_marker_present = "Reasoning process:" in assistant_msg
        print(f"Example {i} has reasoning marker: {reasoning_marker_present}")
        
        # Tokenize and decode to check preservation
        tokens = tokenizer.encode(assistant_msg)
        decoded = tokenizer.decode(tokens)
        
        # Check if markers survived tokenization
        reasoning_preserved = "Reasoning process:" in decoded
        print(f"Example {i} preserved reasoning after tokenization: {reasoning_preserved}")
        
        # Print the structure
        print(f"Structure check for example {i}:")
        lines = decoded.split("\n")
        for j, line in enumerate(lines[:10]):  # First 10 lines
            print(f"  Line {j}: {line[:50]}..." if len(line) > 50 else f"  Line {j}: {line}")
        print("  ...\n")

# Run the check
check_tokenization(reasoning_dataset, tokenizer)

Example 0 has reasoning marker: True
Example 0 preserved reasoning after tokenization: True
Structure check for example 0:
  Line 0: I'll convert this metadata to UNIMARC/XML.
  Line 1: 
  Line 2: Reasoning process:To transform the provided biblio...
  Line 3: 
  Line 4: 1. **Control fields** (these are often generated/m...
  Line 5: 
  Line 6:    - `<controlfield tag="001">` — Record identifie...
  Line 7:    - `<controlfield tag="003">` — Control number i...
  Line 8:    - `<controlfield tag="005">` — Date and time of...
  Line 9:    - `<controlfield tag="035">` — Various system i...
  ...

Example 1 has reasoning marker: True
Example 1 preserved reasoning after tokenization: True
Structure check for example 1:
  Line 0: I'll convert this metadata to UNIMARC/XML.
  Line 1: 
  Line 2: Reasoning process:Let's analyze the transformation...
  Line 3: 
  Line 4: 1. **Title and subtitle**
  Line 5:    - Metadata:
  Line 6:      - Title: "Mémo visuel d'agronomie"
  Line 7:      - Subtitle: 

In [17]:
# Create a custom data collator that logs what it's doing
class DebugCompletionCollator:
    def __init__(self, tokenizer, debug_samples=3):
        self.tokenizer = tokenizer
        self.debug_samples = debug_samples
        self.samples_seen = 0
        
    def __call__(self, examples):
        # Convert the examples to a batch
        batch = self.tokenizer.pad(examples, return_tensors="pt")
        
        # Find the assistant token position
        assistant_token_id = self.tokenizer.encode("<|assistant|>", add_special_tokens=False)[-1]
        # For debugging - if assistant token isn't found, try others
        if assistant_token_id not in batch["input_ids"][0]:
            possible_tokens = ["<|assistant|>","<assistant>", "[assistant]", "assistant:", "assistant\n","ASSISTANT:"]
            for token in possible_tokens:
                try:
                    assistant_token_id = self.tokenizer.encode(token, add_special_tokens=False)[-1]
                    if assistant_token_id in batch["input_ids"][0]:
                        print(f"Found alternative assistant token: {token}")
                        break
                except:
                    continue
        
        # Initialize labels with -100 (ignore in loss)
        batch["labels"] = batch["input_ids"].clone()
        
        # For each sequence in the batch
        for i, input_seq in enumerate(batch["input_ids"]):
            # Try to find assistant token
            assistant_pos = -1
            for pos in range(len(input_seq)):
                if input_seq[pos] == assistant_token_id:
                    assistant_pos = pos
                    break
            
            # If we found the assistant token
            if assistant_pos >= 0:
                # Mask tokens before assistant (set to -100)
                batch["labels"][i, :assistant_pos+1] = -100
            else:
                # If no assistant token found, warn and use heuristic
                if self.samples_seen < self.debug_samples:
                    print(f"Warning: No assistant token found in sequence {i}!")
                    decoded = self.tokenizer.decode(input_seq)
                    print(f"Sample tokens: {decoded[:100]}...")
                
                # Fallback: assume the first half is prompt, second half is completion
                midpoint = len(input_seq) // 2
                batch["labels"][i, :midpoint] = -100
            
            # Debug information for the first few samples
            if self.samples_seen < self.debug_samples:
                # Count how many tokens contribute to the loss
                active_tokens = (batch["labels"][i] != -100).sum().item()
                total_tokens = len(input_seq)
                
                print(f"Sample {self.samples_seen}: {active_tokens}/{total_tokens} tokens used for training")
                
                # Decode a snippet of what's being trained on
                if assistant_pos >= 0:
                    train_text = self.tokenizer.decode(input_seq[assistant_pos+1:])
                    print(f"Training on: {train_text[:100]}...")
                    
                    # Check if reasoning tags are in the training portion
                    if "<think>" in train_text or "Reasoning" in train_text:
                        print("✓ Reasoning traces found in training portion")
                    else:
                        print("✗ No reasoning traces found in training portion!")
                
                self.samples_seen += 1
        
        return batch

# Instead of using Unsloth's default collator, use our debug collator
data_collator = DebugCompletionCollator(tokenizer)

In [18]:
# Add a callback to periodically check what's being generated during training
class ReasoningMonitorCallback(TrainerCallback):
    def __init__(self, tokenizer, eval_dataset, check_every=500):
        self.tokenizer = tokenizer
        self.eval_dataset = eval_dataset
        self.check_every = check_every
        
    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % self.check_every == 0:
            model = kwargs.get("model")
            
            # Select a sample
            sample_idx = state.global_step % len(self.eval_dataset)
            sample = self.eval_dataset[sample_idx]
            
            # Create input prompt
            #system = [msg["content"] for msg in sample["messages"] if msg["role"] == "system"][0]
            #user = [msg["content"] for msg in sample["messages"] if msg["role"] == "user"][0]
            
            #prompt = f"<|system|>\n{system}\n<|user|>\n{user}\n<|assistant|>\n"
            prompt = sample["text"].split("assistant\n")[0]
            inputs = self.tokenizer(prompt, return_tensors="pt").to(model.device)
            
            # Generate output
            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=512,
                    temperature=0.7,
                    do_sample=True
                )
            
            # Decode output
            generated = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            assistant_part = generated.split("<|assistant|>\n")[-1]
            
            # Check for reasoning
            has_reasoning = (
                "Reasoning process:" in assistant_part or
                "reasoning:" in assistant_part.lower() or
                "step by step:" in assistant_part.lower()
            )
            
            print(f"\nStep {state.global_step} generation check:")
            print(f"Has reasoning: {'✓' if has_reasoning else '✗'}")
            print(f"Generated (truncated):\n{assistant_part[:300]}...\n")

reasoning_monitor = ReasoningMonitorCallback(
    tokenizer=tokenizer,
    eval_dataset=eval_dataset,
    check_every=5
)

In [20]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    label_names=["labels"],
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 1, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        #seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)
trainer.add_callback(reasoning_monitor)

Unsloth: Tokenizing ["text"]: 100%|██████████| 2/2 [00:00<00:00, 49.16 examples/s]


In [21]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 18 | Num Epochs = 2 | Total steps = 30
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 1 x 1) = 1
 "-____-"     Trainable parameters = 4,325,376/5,000,000,000 (0.09% trained)


Step,Training Loss
1,2.240900
2,2.612500
3,2.645000
4,2.589500
5,2.571700
6,2.609700
7,2.701700
8,2.288800
9,1.740700
10,2.413100



Step 5 generation check:
Has reasoning: ✓
Generated (truncated):
system
You are an expert bibliographic cataloger specializing in UNIMARC format. 
Please first thinks about the reasoning process in the mind and then provides the user with the answer.

Reasoning process: [YOUR REASONING STEPS HERE]
Final UNIMARC/XML record: [THE RESULTING UNIAMRC/XML RECORD HERE]
...


Step 10 generation check:
Has reasoning: ✓
Generated (truncated):
system
You are an expert bibliographic cataloger specializing in UNIMARC format. 
Please first thinks about the reasoning process in the mind and then provides the user with the answer.

Reasoning process: [YOUR REASONING STEPS HERE]
Final UNIMARC/XML record: [THE RESULTING UNIAMRC/XML RECORD HERE]
...


Step 15 generation check:
Has reasoning: ✓
Generated (truncated):
system
You are an expert bibliographic cataloger specializing in UNIMARC format. 
Please first thinks about the reasoning process in the mind and then provides the user with the answer.

Rea

In [22]:
user_metadata = """
Title: La dynastie des Ming
Author: Jean-Pierre Duteil
Editor/Director: sous la direction de Jean Heuclin
Author details:
- Duteil, Jean-Pierre (1952-...)
- Heuclin, Jean (1949-...), historien
Edition: Nouvelle édition
Publisher: Ellipses
Place of publication: Paris
Year: 2024
ISBN: 978-2-340-08969-3 (note: also 9782340089693)
Language: French
Country of publication: France
Physical description: 1 volume (287 pages), includes cartes, plans, cover illustrated in color, 18 cm
Bibliography: Bibliogr. p. 279-286
Abstract/Notes:
Les Ming, une dynastie brillante dominée par les Han installés à Pékin à l’époque du Quattrocento et de la Renaissance en Europe ; les deux civilisations devaient se rencontrer par une expansion maritime similaire, une découverte inquiète et remplie des mystères orientaux. Le fondateur Hongwu (1368-1398) à l’instar de notre Henri IV, tournant le dos à l’influence nomade mongole, établit une société rurale autosuffisante dont l’essor des productions céréalières alimenta toute une économie marchande et urbaine. Des projets grandioses virent le jour comme le Grand Canal, la Grande Muraille ou la Cité interdite avec une puissante marine et une armée de métier. Tout un système administratif de fonctionnaires recrutés par concours sur leurs mérites accompagne une remarquable création littéraire, calligraphique et artistique facilitée par l’impression de livres. Cette magnifique construction tomba progressivement sous l’emprise sclérosante de ces mêmes « lettrés » trop empreints d’un confucianisme qui résistait mal à la critique de la politique gouvernementale et au comportement extravagant des empereurs. Des révoltes se firent jour durant le dernier siècle de la dynastie marquée par une succession de catastrophes naturelles et d’épidémies qui ruinèrent l’une des périodes les plus riches de l’histoire de l’Empire du Milieu.
Source of the abstract/notes: 4e de couverture (back cover)
Subject/Keywords:
- Chine
- Dynastie des Ming (1368-1644)
Classification: 951.02 (subject classification)
Publication history note: Paru précédemment en 2016
Additional edition note: La 1ère édition de ce livre a été publiée en grand format en 2016 dans la collection "Biographies et mythes historiques" dirigée par Stéphane Curveiller
Collection/Series: Collection "Biographies et mythes historiques" (inferred from note about first edition)
"""

messages = [
    {"role": "system", "content": system_prompt},
    {"role" : "user", "content" : user_metadata}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    #enable_thinking = False, # Disable thinking
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 4096, # Increase for longer outputs!
    temperature = 0.6, top_p = 0.95, top_k = 20, # For thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

User's response matches the provided information. Here is the final UNIMARC/XML record:

```
<?xml version="1.0"?>
<record xmlns="http://www.umi.edu/xml/ns/uniamc-xml-format 2.0#" version="4">
    <metadata>
        <identifier identifierTarget="_dcid01" scheme="http://purl.org/dc/elements/1.1/id"/>
        <identifier identifierTarget="_id01" scheme="http://purl.org/dc/elements/1.1/description"/>
        <identifier identifierTarget="_acmid01" scheme="http://purl.org/ukpl/acmid"/>
        <identifier identifierTarget="_isbn01" scheme="https://bibliothèques.univ-amonthe.fr/zbw/nom/isbn/9782340089693"/>
        <identifier identifierTarget="_source01" scheme="urn:iadsc:unimaubiblio:unamarc"/>
        <identifier identifierTarget="_author01" scheme="http://purl.org/auth/identifier/author"/>
        <identifier identifierTarget="_editor01" scheme="http://purl.org/auth/identifier/editor"/>
        <identifier identifierTarget="_publisher01" scheme="http://purl.org/auth/identifier/publisher

KeyboardInterrupt: 